In [1]:
import keras 
import pandas as pd
import matplotlib.pyplot as plt 
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image 
import os
import numpy as np
import math
import dlib
import cv2
import easydict
from keras.models import Sequential
from keras import layers
from keras.utils import to_categorical
import csv
import pandas as pd
from sklearn.model_selection import train_test_split

C:\Users\deepu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
args={"shape_predictor":"shape_predictor_68_face_landmarks.dat"}

In [3]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor(args["shape_predictor"])

In [4]:
def rect_to_bb(rect):
    x=rect.left()
    y=rect.top()
    w=rect.right()-x
    h=rect.bottom()-y
    return (x,y,w,h)

In [5]:
def shape_to_np(shape,dtype='int'):
    coords=np.zeros((68,2),dtype=dtype)
    for i in range(0,68):
        coords[i]=(shape.part(i).x,shape.part(i).y)
    return coords

In [6]:
def preprocess_dis(dis_array):
    x=dis_array-np.mean(dis_array)
    if np.std(dis_array)!=0:
        x/=np.std(dis_array)
    return x

In [7]:
def convert_points_to_distance(points):

    mid_x=(points[14,0]+points[18,0])/2
    mid_y=(points[14,1]+points[18,1])/2
    mid_point=np.array([mid_x,mid_y])
    points=(points-mid_point)**2
    distances_array=np.sqrt((points[:,0]+points[:,1]))
    
    return preprocess_dis(distances_array)
    

In [8]:
def face_landmarks(image):
    #all_images=[]
    #for image in images:
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    rects=detector(gray,1)
    shape=np.zeros((20,2))
    for rect in rects:
        shape=predictor(gray,rect)
        shape=shape_to_np(shape)
        shape=shape[48:68]
    #all_images.append(shape)

    return shape   

In [9]:
def empty_distance(number):
    shape=np.zeros((20,number))
    return shape

In [ ]:
#labels=[]
#lip_data=[]

train_dir="DataRest"
for person in os.listdir(train_dir):
    person_word_dir=os.path.join(train_dir,person,'words')
    for label in os.listdir(person_word_dir):
        labels_dir=os.path.join(person_word_dir,label)
        for repeat in os.listdir(labels_dir):

            word_dir=os.path.join(labels_dir,repeat)
            print("currently in %", word_dir)
            images_array=[]
            num_images=0
            for image_dir in os.listdir(word_dir):
                if num_images==10:
                    break
                image=cv2.imread(os.path.join(word_dir,image_dir))
                points=face_landmarks(image) 
                #points=convert_points_to_distance(points)
                #if points is not None:
                images_array.append(points)

                num_images=num_images+1
            if num_images<10:
                images_array=np.append(images_array,empty_distance(10-num_images))
            lip_data=np.append(lip_data,np.array(images_array))
            labels.append(int(label)-1)
lip_data=lip_data.reshape(1500,10,20)  

In [268]:
lip_data=np.load('all_words.npy')
labels=np.load('all_words_labels.npy')

In [269]:
diff_lip_data=y=lip_data.T.reshape(20,1500,10,1)

In [270]:
labels_hot=to_categorical(labels)

In [271]:
X_train,X_test,Y_train,Y_test=train_test_split(lip_data,labels_hot,test_size=0.2)


In [272]:
X_train=X_train.T.reshape(20,1200,10,1)
X_test=X_test.T.reshape(20,300,10,1)

In [273]:
from keras.regularizers import l1,l2

In [344]:
"""model=Sequential()
# Try different combinations and improve the accuracy
model.add(layers.LSTM(512,activation='tanh',return_sequences=True,input_shape=(10,20)))
#model.add(layers.BatchNormalization())
model.add(layers.LSTM(128,activation='tanh',return_sequences=True,activity_regularizer=l2(0.1)))
#model.add(layers.BatchNormalization())
model.add(layers.LSTM(64,activation='tanh'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32))
model.add(layers.LeakyReLU(0.1))

#model.add(layers.Dense(64,activation='relu'))

model.add(layers.Dense(10,activation="softmax"))"""
model = Sequential()
#model.add(layers.LSTM(,activation='relu',return_sequences=True,input_shape=(None,20)))

model.add(layers.LSTM(256,activation='tanh',return_sequences=True,input_shape=(None,1)))

model.add(layers.LSTM(128,return_sequences=True,activation='tanh'))
model.add(layers.ActivityRegularization(l2=0.01))
model.add(layers.LSTM(64,return_sequences=True,activation='tanh'))
model.add(layers.LSTM(32,activation='tanh'))
model.add(layers.Dense(16,activation='tanh'))
model.add(layers.Dense(10,activation="softmax"))

In [345]:
def lr(epoch):
    return 0.001*(0.99)**epoch

In [346]:
callback_lists=[keras.callbacks.LearningRateScheduler(lr)]

In [347]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_95 (LSTM)               (None, None, 256)         264192    
_________________________________________________________________
lstm_96 (LSTM)               (None, None, 128)         197120    
_________________________________________________________________
activity_regularization_17 ( (None, None, 128)         0         
_________________________________________________________________
lstm_97 (LSTM)               (None, None, 64)          49408     
_________________________________________________________________
lstm_98 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_55 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_56 (Dense)             (None, 10)                170       
Total para

In [348]:
X_train.shape

(20, 1200, 10, 1)

In [349]:
model.compile(loss="categorical_crossentropy",
             metrics=['accuracy'],
             
             optimizer='rmsprop',
             )

In [350]:
models=[]
for i in range(20):
    models.append(model)

In [353]:

for i in range(2,20):
    models[i].fit(X_train[i],Y_train,batch_size=32,epochs=25,validation_data=(X_test[i],Y_test))

Train on 1200 samples, validate on 300 samples
Epoch 1/25
1200/1200 [==============================] - 8s 7ms/step - loss: 2.4024 - acc: 0.0925 - val_loss: 2.3440 - val_acc: 0.0900
Epoch 2/25
1200/1200 [==============================] - 8s 6ms/step - loss: 2.3311 - acc: 0.0867 - val_loss: 2.3227 - val_acc: 0.0800
Epoch 3/25
1200/1200 [==============================] - 8s 6ms/step - loss: 2.3126 - acc: 0.1050 - val_loss: 2.3317 - val_acc: 0.1067
Epoch 4/25
1200/1200 [==============================] - 8s 7ms/step - loss: 2.3152 - acc: 0.0942 - val_loss: 2.3104 - val_acc: 0.0967
Epoch 5/25
1200/1200 [==============================] - 8s 7ms/step - loss: 2.3123 - acc: 0.1017 - val_loss: 2.3067 - val_acc: 0.1100
Epoch 6/25
1200/1200 [==============================] - 8s 6ms/step - loss: 2.3086 - acc: 0.1017 - val_loss: 2.3136 - val_acc: 0.0867
Epoch 7/25
1200/1200 [==============================] - 17s 14ms/step - loss: 2.3096 - acc: 0.1008 - val_loss: 2.3151 - val_acc: 0.0867
Epoch 8/25
12

1200/1200 [==============================] - 14s 12ms/step - loss: 2.3002 - acc: 0.0975 - val_loss: 2.3236 - val_acc: 0.0700
Epoch 11/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.3002 - acc: 0.1067 - val_loss: 2.3159 - val_acc: 0.0900
Epoch 12/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.3009 - acc: 0.1117 - val_loss: 2.3232 - val_acc: 0.0933
Epoch 13/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2969 - acc: 0.1092 - val_loss: 2.3151 - val_acc: 0.0933
Epoch 14/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2992 - acc: 0.1050 - val_loss: 2.3182 - val_acc: 0.1167
Epoch 15/25
1200/1200 [==============================] - 13s 11ms/step - loss: 2.2998 - acc: 0.1158 - val_loss: 2.3230 - val_acc: 0.1000
Epoch 16/25
1200/1200 [==============================] - 13s 11ms/step - loss: 2.2934 - acc: 0.1158 - val_loss: 2.3113 - val_acc: 0.1067
Epoch 17/25
1200/1200 [==============================

1200/1200 [==============================] - 13s 11ms/step - loss: 2.2864 - acc: 0.1225 - val_loss: 2.3237 - val_acc: 0.1100
Epoch 20/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2840 - acc: 0.1225 - val_loss: 2.3338 - val_acc: 0.0933
Epoch 21/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2797 - acc: 0.1267 - val_loss: 2.3270 - val_acc: 0.0933
Epoch 22/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2810 - acc: 0.1267 - val_loss: 2.3291 - val_acc: 0.1167
Epoch 23/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2810 - acc: 0.1233 - val_loss: 2.3192 - val_acc: 0.1000
Epoch 24/25
1200/1200 [==============================] - 13s 11ms/step - loss: 2.2796 - acc: 0.1300 - val_loss: 2.3291 - val_acc: 0.0733
Epoch 25/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2780 - acc: 0.1267 - val_loss: 2.3321 - val_acc: 0.0867
Train on 1200 samples, validate on 300 samples
Epoch 

1200/1200 [==============================] - 13s 11ms/step - loss: 2.2942 - acc: 0.1167 - val_loss: 2.3433 - val_acc: 0.0933
Epoch 4/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2915 - acc: 0.1167 - val_loss: 2.3165 - val_acc: 0.1200
Epoch 5/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2879 - acc: 0.1142 - val_loss: 2.3314 - val_acc: 0.1100
Epoch 6/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2828 - acc: 0.1267 - val_loss: 2.3278 - val_acc: 0.0900
Epoch 7/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2795 - acc: 0.1300 - val_loss: 2.3251 - val_acc: 0.0900
Epoch 8/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2763 - acc: 0.1317 - val_loss: 2.3466 - val_acc: 0.0900
Epoch 9/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2745 - acc: 0.1467 - val_loss: 2.3431 - val_acc: 0.1067
Epoch 10/25
1200/1200 [==============================] - 14

1200/1200 [==============================] - 14s 12ms/step - loss: 2.2775 - acc: 0.1308 - val_loss: 2.3213 - val_acc: 0.1267
Epoch 13/25
1200/1200 [==============================] - 15s 13ms/step - loss: 2.2734 - acc: 0.1242 - val_loss: 2.3168 - val_acc: 0.1367
Epoch 14/25
1200/1200 [==============================] - 13s 11ms/step - loss: 2.2731 - acc: 0.1467 - val_loss: 2.3294 - val_acc: 0.1133
Epoch 15/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2693 - acc: 0.1358 - val_loss: 2.3260 - val_acc: 0.1133
Epoch 16/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2636 - acc: 0.1533 - val_loss: 2.3283 - val_acc: 0.1200
Epoch 17/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2617 - acc: 0.1517 - val_loss: 2.3287 - val_acc: 0.1300
Epoch 18/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2540 - acc: 0.1483 - val_loss: 2.3278 - val_acc: 0.1033
Epoch 19/25
1200/1200 [==============================

1200/1200 [==============================] - 14s 12ms/step - loss: 2.2871 - acc: 0.1058 - val_loss: 2.3054 - val_acc: 0.1067
Epoch 22/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2843 - acc: 0.1192 - val_loss: 2.3039 - val_acc: 0.1133
Epoch 23/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2891 - acc: 0.1200 - val_loss: 2.3098 - val_acc: 0.1100
Epoch 24/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2817 - acc: 0.1175 - val_loss: 2.3124 - val_acc: 0.1100
Epoch 25/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2852 - acc: 0.1067 - val_loss: 2.3130 - val_acc: 0.1167
Train on 1200 samples, validate on 300 samples
Epoch 1/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.3032 - acc: 0.1033 - val_loss: 2.3045 - val_acc: 0.1067
Epoch 2/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2988 - acc: 0.1067 - val_loss: 2.3047 - val_acc: 0.1067
Epoch 3/

1200/1200 [==============================] - 14s 11ms/step - loss: 2.2836 - acc: 0.1283 - val_loss: 2.3234 - val_acc: 0.1100
Epoch 6/25
1200/1200 [==============================] - 13s 11ms/step - loss: 2.2712 - acc: 0.1317 - val_loss: 2.3349 - val_acc: 0.0700
Epoch 7/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2653 - acc: 0.1408 - val_loss: 2.3377 - val_acc: 0.0933
Epoch 8/25
1200/1200 [==============================] - 14s 11ms/step - loss: 2.2598 - acc: 0.1408 - val_loss: 2.3369 - val_acc: 0.0767
Epoch 9/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2558 - acc: 0.1525 - val_loss: 2.3494 - val_acc: 0.0800
Epoch 10/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2492 - acc: 0.1500 - val_loss: 2.3371 - val_acc: 0.1100
Epoch 11/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2478 - acc: 0.1525 - val_loss: 2.3609 - val_acc: 0.0900
Epoch 12/25
1200/1200 [==============================] - 

1200/1200 [==============================] - 14s 12ms/step - loss: 2.2431 - acc: 0.1308 - val_loss: 2.3651 - val_acc: 0.1100
Epoch 15/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2426 - acc: 0.1250 - val_loss: 2.3803 - val_acc: 0.1300
Epoch 16/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2359 - acc: 0.1225 - val_loss: 2.4040 - val_acc: 0.1033
Epoch 17/25
1200/1200 [==============================] - 13s 11ms/step - loss: 2.2385 - acc: 0.1358 - val_loss: 2.3532 - val_acc: 0.1100
Epoch 18/25
1200/1200 [==============================] - 14s 12ms/step - loss: 2.2322 - acc: 0.1333 - val_loss: 2.3692 - val_acc: 0.1200
Epoch 19/25
1200/1200 [==============================] - 12s 10ms/step - loss: 2.2327 - acc: 0.1350 - val_loss: 2.3589 - val_acc: 0.1233
Epoch 20/25
1200/1200 [==============================] - 11s 9ms/step - loss: 2.2284 - acc: 0.1450 - val_loss: 2.3233 - val_acc: 0.1133
Epoch 21/25
1200/1200 [==============================]

In [288]:
cv2.createButton()


TypeError: Required argument 'buttonName' (pos 1) not found

In [341]:
from keras.models import load_model
model2=load_model('minor2.h5')

In [474]:
models.evaluate(X_test,Y_test)

AttributeError: 'list' object has no attribute 'evaluate'

In [152]:
model.save('minor_me.h5')

In [476]:
prediction=np.zeros((10))
for i in range(20):
    prediction=prediction+models[i].predict(X_test[i])#+models[2].predict(X_test[2][0][None])

In [489]:
(np.argmax(Y_test,axis=1)==np.argmax(prediction,axis=1)).sum()/300
#Y_test.shape

0.09333333333333334

array([1.11656657, 1.39052836, 2.742844  , 3.22573817, 1.6841809 ,
       3.58336178, 1.45373369, 1.04901264, 1.77854133, 1.97549289])